# Using DeepEval for AWS Bedrock LLM evaluation

## Build an agent with AWS Strands Agent

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. We will quick create the Strands agent first.

**Prerequisites**

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [1]:
%pip install strands-agents strands-agents-tools boto3 botocore -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Restart kernel (only works on Linux)
import os

os._exit(00)

**Set Your AWS Credentials**
There are multiple ways to set your AWS Credentials depending on your environment.

In [ ]:
session = boto3.Session(
    region_name="us-east-1",
    aws_access_key_id="<YOUR_ACCESS_KEY_ID>",
    aws_secret_access_key="<YOUR_SECRET_ACCESS_KEY>",
    aws_session_token="<YOUR_SESSION_TOKEN>",
)

**Custom Tool Demonstration**

In [1]:
from strands import Agent, tool
from strands_tools import calculator, current_time, python_repl,file_read,shell,file_write
import nest_asyncio

# Apply nest_asyncio at the start
nest_asyncio.apply()

@tool
def word_count(text: str) -> int:
    """Count words in text.

    This docstring is used by the LLM to understand the tool's purpose.
    """
    return len(text.split())


**Define an agent**

In [2]:
agent = Agent(tools=[calculator, current_time, python_repl, word_count,file_read,shell,file_write],model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")

In [11]:
message = "You are a helpful assistant that provides concise responses. Help me to read letter.txt file, count the total number of words."

results = agent(message)

I'll help you read the letter.txt file and count the total number of words.
Tool #3: file_read


╔════ 📄 letter.txt ═════╗
║                        ║
║    1 YOU ARE THE BEST  ║
║                        ║
╚════════════════════════╝


Tool #4: word_count
The letter.txt file contains 4 words.

**See the excution and tool use results**

In [12]:
for m in agent.messages:
    for content in m["content"]:
        if "toolUse" in content:
            print("Tool Use:")
            tool_use = content["toolUse"]
            print("\tToolUseId: ", tool_use["toolUseId"])
            print("\tname: ", tool_use["name"])
            print("\tinput: ", tool_use["input"])
        if "toolResult" in content:
            print("Tool Result:")
            tool_result = m["content"][0]["toolResult"]
            print("\tToolUseId: ", tool_result["toolUseId"])
            print("\tStatus: ", tool_result["status"])
            print("\tContent: ", tool_result["content"])
            print("=======================")

Tool Use:
	ToolUseId:  tooluse_sN51vJtsQGCD-_w1LwTIYw
	name:  file_read
	input:  {'path': 'letter.txt', 'mode': 'view'}
Tool Result:
	ToolUseId:  tooluse_sN51vJtsQGCD-_w1LwTIYw
	Status:  success
	Content:  [{'text': 'Content of letter.txt:\nYOU ARE THE BEST'}]
Tool Use:
	ToolUseId:  tooluse_nRdajCV2SieeqQWyp-v5Sw
	name:  word_count
	input:  {'text': 'YOU ARE THE BEST'}
Tool Result:
	ToolUseId:  tooluse_nRdajCV2SieeqQWyp-v5Sw
	Status:  success
	Content:  [{'text': '4'}]
Tool Use:
	ToolUseId:  tooluse_QqicIo7PTHy1iL8bJFfrWw
	name:  file_read
	input:  {'path': 'letter.txt', 'mode': 'view'}
Tool Result:
	ToolUseId:  tooluse_QqicIo7PTHy1iL8bJFfrWw
	Status:  success
	Content:  [{'text': 'Content of letter.txt:\nYOU ARE THE BEST'}]
Tool Use:
	ToolUseId:  tooluse_UsAow-quSCuUe3kRTKYikg
	name:  word_count
	input:  {'text': 'YOU ARE THE BEST'}
Tool Result:
	ToolUseId:  tooluse_UsAow-quSCuUe3kRTKYikg
	Status:  success
	Content:  [{'text': '4'}]


## Use DeepEval for tool use evaluation

Now, we have use Strands Agents to build an assistant agent with five tools. First we use DeepEval for tool use evaluation.

**Tool correctness**


Tool Correctness assesses whether an agent’s tool-calling behavior aligns with expectations by verifying that all required tools were correctly called. Unlike most LLM evaluation metrics, the Tool Correctness metric is a deterministic measure and not an LLM-judge.


In [13]:
from deepeval.models.llms.amazon_bedrock_model import AmazonBedrockModel
import nest_asyncio

# Apply nest_asyncio at the start
nest_asyncio.apply()

# Initialize the Bedrock model (e.g., Claude)
model = AmazonBedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-east-1"
)


In [15]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ToolCorrectnessMetric
from deepeval.test_case import LLMTestCase, ToolCall,ToolCallParams
from deepeval import evaluate
from extract_tool_calls import extract_tool_calls_from_strands



tool_calls, final_text = extract_tool_calls_from_strands(agent.messages)

# Debug tool calls
print("\nDebug - Tool calls:", tool_calls)

# Create test case with string output
test_case = LLMTestCase(
    input=message,
    actual_output="The file contains 4 words: YOU, ARE, THE, BEST",
    tools_called=tool_calls,
    expected_tools=[ToolCall(name="file_read"), ToolCall(name="word_count")]
)

task_Correctness_metric = ToolCorrectnessMetric()

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[task_Correctness_metric],
)


Created ToolCall: {'name': 'file_read', 'description': 'Tool used in the conversation: file_read', 'reasoning': None, 'output': ["{'text': 'Content of letter.txt:\\nYOU ARE THE BEST'}"], 'input_parameters': {'path': 'letter.txt', 'mode': 'view'}}
Created ToolCall: {'name': 'word_count', 'description': 'Tool used in the conversation: word_count', 'reasoning': None, 'output': ["{'text': '4'}"], 'input_parameters': {'text': 'YOU ARE THE BEST'}}
Created ToolCall: {'name': 'file_read', 'description': 'Tool used in the conversation: file_read', 'reasoning': None, 'output': ["{'text': 'Content of letter.txt:\\nYOU ARE THE BEST'}"], 'input_parameters': {'path': 'letter.txt', 'mode': 'view'}}
Created ToolCall: {'name': 'word_count', 'description': 'Tool used in the conversation: word_count', 'reasoning': None, 'output': ["{'text': '4'}"], 'input_parameters': {'text': 'YOU ARE THE BEST'}}
Created tool calls:
  - ToolCall: file_read
  - ToolCall: word_count
  - ToolCall: file_read
  - ToolCall: w

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:00, 139.99test case/s]



Metrics Summary

  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: All expected tools ['file_read', 'word_count'] were called (order not considered)., error: None)

For test case:

  - input: You are a helpful assistant that provides concise responses. Help me to read letter.txt file, count the total number of words.
  - actual output: The file contains 4 words: YOU, ARE, THE, BEST
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Tool Correctness: 100.00% pass rate




✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Tool Correctness', threshold=0.5, success=True, score=1.0, reason="All expected tools ['file_read', 'word_count'] were called (order not considered).", strict_mode=False, evaluation_model=None, error=None, evaluation_cost=None, verbose_logs='Expected Tools:\n[\n    ToolCall(\n        name="file_read"\n    ),\n    ToolCall(\n        name="word_count"\n    )\n] \n \nTools Called:\n[\n    ToolCall(\n        name="file_read",\n        description="Tool used in the conversation: file_read",\n        input_parameters={\n            "path": "letter.txt",\n            "mode": "view"\n        },\n        output=["{\'text\': \'Content of letter.txt:\\\\nYOU ARE THE BEST\'}"]\n    ),\n    ToolCall(\n        name="word_count",\n        description="Tool used in the conversation: word_count",\n        input_parameters={\n            "text": "YOU ARE THE BEST"\n        },\n        output=["{\'t

**Tool Efficiency**


Equally important to tool correctness is tool efficiency. Inefficient tool-calling patterns can increase response times, frustrate users, and significantly raise operational costs.


Let’s explore how tool efficiency can be evaluated:

* Redundant Tool --  Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome. This can be calculated as the percentage of unnecessary tools relative to the total number of tool invocations.
* Tool Frequency -- evaluates whether tools are being called more often than necessary. This method penalizes tools that exceed a predefined threshold for the number of calls required to complete a task (many times this is just 1).

In [18]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams


In [24]:


# Create test case
test_case = LLMTestCase(
    input=message,
    actual_output=final_text,
    tools_called=tool_calls
)


# Create G-Eval metric for tool efficiency with Bedrock model
g_eval_metric = GEval(
    name="Tool Efficiency",
    criteria="""
Determine whether the tool effectively be used.
Redundant Tool Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome.
Tool Frequency evaluates whether tools are being called more often than necessary.
""",
    threshold=0.7,  # Set a reasonable threshold for tool efficiency
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.TOOLS_CALLED
    ],
    model=model  # Explicitly pass the Bedrock model
)

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[g_eval_metric],
)


✨ You're running DeepEval's latest Tool Efficiency (GEval) Metric! (using 
us.anthropic.claude-3-7-sonnet-20250219-v1:0, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:15, 15.52s/test case]



Metrics Summary

  - ❌ Tool Efficiency (GEval) (score: 0.4, threshold: 0.7, strict: False, evaluation model: us.anthropic.claude-3-7-sonnet-20250219-v1:0, reason: The assistant correctly used the necessary tools (file_read and word_count) to complete the task of counting words in letter.txt, and the output accurately reports 4 words. However, there are significant inefficiencies: both tools were called twice with identical parameters, producing duplicate results. The task could have been completed with just one file_read call followed by one word_count call, making half of the tool calls redundant., error: None)

For test case:

  - input: You are a helpful assistant that provides concise responses. Help me to read letter.txt file, count the total number of words.
  - actual output: {'text': 'The letter.txt file contains 4 words.'}
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Tool Efficiency (GEval): 0.00% pass rate




✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Tool Efficiency (GEval)', threshold=0.7, success=False, score=0.4, reason='The assistant correctly used the necessary tools (file_read and word_count) to complete the task of counting words in letter.txt, and the output accurately reports 4 words. However, there are significant inefficiencies: both tools were called twice with identical parameters, producing duplicate results. The task could have been completed with just one file_read call followed by one word_count call, making half of the tool calls redundant.', strict_mode=False, evaluation_model='us.anthropic.claude-3-7-sonnet-20250219-v1:0', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\n\nDetermine whether the tool effectively be used.\nRedundant Tool Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome.\nTool Frequency evaluates whether

## Evaluating Agentic Workflows

**Task Completion**

A critical metric for assessing agent workflows is Task Completion (also known as task success or goal accuracy). This metric measures how effectively an LLM agent completes a user-given task. 

However, in real-world applications, agents are often required to perform a diverse set of tasks—many of which may lack predefined ground-truth datasets.DeepEval’s Task Completion metric addresses these challenges by leveraging LLMs to:

* Determine the task from the user’s input.
* Analyze the reasoning steps, tool usage, and final response to assess whether the task was successfully completed.

In [26]:
from deepeval.metrics import TaskCompletionMetric

# Create test case
test_case = LLMTestCase(
    input=message,
    actual_output=final_text,
    tools_called=tool_calls
)


task_completion_metric = TaskCompletionMetric(model=model)

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[task_completion_metric],
)

✨ You're running DeepEval's latest Task Completion Metric! (using us.anthropic.claude-3-7-sonnet-20250219-v1:0, 
strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:05,  5.22s/test case]



Metrics Summary

  - ✅ Task Completion (score: 1.0, threshold: 0.5, strict: False, evaluation model: us.anthropic.claude-3-7-sonnet-20250219-v1:0, reason: The actual outcome perfectly achieves the user's goal. The system successfully read the letter.txt file and accurately counted the total number of words (4) in the file content 'YOU ARE THE BEST'., error: None)

For test case:

  - input: You are a helpful assistant that provides concise responses. Help me to read letter.txt file, count the total number of words.
  - actual output: {'text': 'The letter.txt file contains 4 words.'}
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Task Completion: 100.00% pass rate




✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Task Completion', threshold=0.5, success=True, score=1.0, reason="The actual outcome perfectly achieves the user's goal. The system successfully read the letter.txt file and accurately counted the total number of words (4) in the file content 'YOU ARE THE BEST'.", strict_mode=False, evaluation_model='us.anthropic.claude-3-7-sonnet-20250219-v1:0', error=None, evaluation_cost=0.0, verbose_logs="User Goal: Read letter.txt file and count the total number of words. \n \nTask Outcome: The system read the letter.txt file which contained 'YOU ARE THE BEST' and counted 4 words in the file.")], conversational=False, multimodal=False, input='You are a helpful assistant that provides concise responses. Help me to read letter.txt file, count the total number of words.', actual_output="{'text': 'The letter.txt file contains 4 words.'}", expected_output=None, context=None, retrieval_context=None